## Import & Environment Setting

In [ ]:
import torch
from torch import autograd
from torch.utils.data import DataLoader

import json
import gc
import numpy as np

from utils.dataset import DetectionFolder
from utils.model import YoloV3, YoloLoss, PostProcessor

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dtype = torch.float

## Load Config

In [ ]:
with open("./config/config.json", "r") as config_file:
    main_config = json.load(config_file)

try:
    model_config = main_config['model']
    loss_config = main_config['loss']
    train_config = main_config['train']
except NameError:
    assert False, ('Failed to load config file')
except KeyError:
    assert False, ('Failed to find key on config file')

In [ ]:
model_config['device'] = device
model_config['dtype'] = dtype
model_config['attrib_count'] = 5 + model_config['class_count']

loss_config['device'] = device
loss_config['dtype'] = dtype
loss_config['attrib_count'] = model_config['attrib_count']


train_config['device'] = device
train_config['dtype'] = dtype

## Build

In [ ]:
model = YoloV3(model_config)
model.to(model_config['device'])
model = model.train()

In [ ]:
loss_func = YoloLoss(loss_config)

In [ ]:
train_context = { }

train_context['dataset'] = DetectionFolder(train_config['train_list'], train_config['train_image'], train_config['train_label'])
train_context['dataloader'] = DataLoader(train_context['dataset'], batch_size = train_config['batch_size'], num_workers = 4)

train_context['epoch'] = 0
train_context['last_checkpoint'] = 0

train_context['lr'] = train_config['init_lr']
train_context['loss_window'] = []


train_context['post_mAp'] = 0.5
train_context['post_iou_threshold'] = 0.8

In [ ]:

lr_func = lambda epoch: train_context['lr']
optimizer = torch.optim.Adam(model.parameters(), lr = train_context['lr'])
#train_context['optimizer'] = torch.optim.SGD(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda = lr_func, last_epoch = -1)

In [ ]:
def train(model, loss_func, optimizer, scheduler, train_context, train_config, epochs):
    
    # set anomaly detection
    torch.autograd.set_detect_anomaly(train_config['use_anomaly_detection'])
        
    # training step
    for _ in range(0, epochs):
        print('epoch : ', train_context['epoch'])
        print('    lr : ', train_context['lr'])
        
        batch_limit = train_context['dataset'].__len__() 
        if train_context['epoch'] > train_config['i_drop_start']:
            drop_count = ((train_context['epoch'] - train_config['i_drop_start']) * train_config['i_drop_per_epoch'])
            batch_limit = max(batch_limit - drop_count, train_config['i_drop_bottom_line'])
        batch_limit = batch_limit // train_config['batch_size']
        print('    batch_limit : ', batch_limit)
        
        losses = []
        for idx, batches in enumerate(train_context['dataloader']):
            if idx > batch_limit:
                break
                
            image = batches['image'].to(train_config['device'], dtype = train_config['dtype'])
            labels = batches['label'].to(train_config['device'], dtype = train_config['dtype'])
            label_len = batches['label_len'].to(train_config['device'], dtype = torch.long)
            
            # forward
            out1, out2, out3 = model(image)
       
            # clear optimizer
            optimizer.zero_grad()
        
            # loss
            loss = loss_func(torch.cat((out1, out2, out3), 1), labels, label_len)
            losses.append(loss.item())
            
            # backward
            loss.backward()
            optimizer.step()
            
            if idx > 30:
                postprocessor = PostProcessor()
            
                tmp = postprocessor.NMS(torch.cat((out1, out2, out3), 1).cpu(), train_context)
                print(tmp)
                tmp = postprocessor.GAS(torch.cat((out1, out2, out3), 1).cpu(), train_context)
                print(tmp)
        
            # cleanup
            del image, labels, label_len
            del out1, out2, out3
            gc.collect()
            torch.cuda.empty_cache()
    
        # update learning rate & scheduler
        avg_loss = np.mean(losses) if len(losses) is not 0 else 0
        train_context['loss_window'].append(avg_loss)
        print('    loss : ', avg_loss)
        
        window_len = len(train_context['loss_window'])
        if (len(train_context['loss_window']) >= train_config['lr_window'] and
            np.mean(train_context['loss_window']) <= np.mean(train_context['loss_window'][-3:])):
            
            print('    window size : ', len(train_context['loss_window']))
            print('    decrease lr to : ', train_context['lr'] * train_config['lr_decay'])
                
            train_context['lr'] = train_context['lr'] * train_config['lr_decay']
            train_context['loss_window'] = []
            
        if len(train_context['loss_window']) > 3 * train_config['lr_window']:
            train_context['loss_window'] = train_context['loss_window'][2 * train_config['lr_window']:]
        
        scheduler.step()
        
        
        # update context
        train_context['dataset'].shuffle()
        train_context['epoch'] += 1
        
        # save model
        if train_context['epoch'] % train_config['checkpoint'] is 0:
            train_context['last_checkpoint'] = train_config['checkpoint']
            torch.save(model, train_config['checkpoint_dir'] + 'model_' + str(train_context['epoch']) + '.dat')
    

## Run

In [ ]:
train(model, loss_func, optimizer, scheduler, train_context, train_config, train_config['epochs'])

In [ ]:
print(train_context)